In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from nltk.tokenize import RegexpTokenizer
import re

In [2]:
# read in combined dataset
df = pd.read_csv('./data/combined.csv')

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
# for idx, row in df.iterrows():
#     print(row['title'])

REQUEST FOR INFORMATION - NEW DESIGN BOOKLET DIE CUTTING INSERT ASSEMBLY
Audiovisual Suite for Large Auditorium - Maxwell AFB
Metrology Equipment Move
INNER INFLATABLE ASSY (LPU-36A/P)
17--CRADLE
43--FILTER ELEMENT,FLUI
61--CABLE ASSEMBLY,SPEC
30--GEAR SET,SPUR,MATCH
53--NUT,SELF-LOCKING,EX
6640--Notice of Intent to Sole Source    
Residual Leukocyte Counters
FIRE DOOR LATCH ASSEMBLIES - USNS BRUNSWICK
61--CABLE ASSEMBLY,SPEC
43--FILTER ELEMENT,FLUI
G001--502-20-3-169-0017 BPA MORTUARY AFFAIRS UNCLAIMED VETERAN BURIAL SERVICE   
69--Chevy 350 Engine Trainers and Start Carts
48--INDICATOR,VALVE OPE
Ad Hoc COVID-19 Analytics and Naval Medical Strategic Planning Process Development
Alternate Care Facility (ACF) at Westchester County Center
53--RECEPTACLE,TURNLOCK
CABLE ASSY, SCSI
Heating System Upgrade, Franklin Falls Project Office, Franklin, NH
PNEUMATIC RUBBER FENDERS
61--POWER SUPPLY
Z--Grand Coulee Dam Left and Right Powerplants G1-G18 Penstock Lining Rehabilitati
43--FILTER,FLUID
Pr

In [34]:
# df['responseDeadLine'].fillna()

dtype('O')

In [5]:
df.head()

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,archiveDate,typeOfSetAsideDescription,typeOfSetAside,responseDeadLine,naicsCode,classificationCode,active,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
0,ffe3c07bab8b4a7d9be583fd4dd96c4a,REQUEST FOR INFORMATION - NEW DESIGN BOOKLET DIE CUTTING INSERT ASSEMBLY,RFI-042720,UNITED STATES GOVERNMENT PUBLISHING OFFICE,UNITED STATES GOVERNMENT PUBLISHING OFFICE,Acquisition Services,2020-04-27,Sources Sought,Sources Sought,auto15,2020-05-27,NaN,NaN,2020-05-12T14:00:00-04:00,333244.0,3610,Yes,NaN,"[{'fax': '', 'type': 'primary', 'email': 'rgraham@gpo.gov', 'phone': '2025120032', 'title': None, 'fullName': 'Rosalyn Graham'}, {'fax': '', 'type': 'secondary', 'email': 'rortega@gpo.gov', 'phone': '2025120803', 'title': None, 'fullName': 'Ronald Ortega'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ffe3c07bab8b4a7d9be583fd4dd96c4a,OFFICE,"{'zipcode': '20401', 'city': 'Washington', 'countryCode': 'USA', 'state': 'DC'}","{'state': {'code': 'DC', 'name': 'District of Columbia'}, 'zip': '20401', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",NaN,https://beta.sam.gov/opp/ffe3c07bab8b4a7d9be583fd4dd96c4a/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=ffe3c07bab8b4a7d9be583fd4dd96c4a&limit=1', 'hreflang': None, 'media': None, 'title': None, 'type': None, 'deprecation': None}]","['https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/bd2b3b5e86474278ae1ea1a163ccbfd7/download?api_key=null&token=', 'https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/774a16e89e4d431e85b0eba1a510a24c/download?api_key=null&token=', 'https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/b62d72d82537413f8537bb19160d49bd/download?api_key=null&token=']"
1,ff5e29951ed14b0386ea3291fa6c573f,Audiovisual Suite for Large Auditorium - Maxwell AFB,FA330020X0001,DEPT OF DEFENSE,DEPT OF THE AIR FORCE,FA3300 42 CONS CC,2020-04-27,Sources Sought,Sources Sought,auto15,2020-05-20,NaN,NaN,2020-05-05T12:00:00-05:00,334310.0,5836,Yes,NaN,"[{'fax': '3349532198', 'type': 'primary', 'email': 'brandon.osborne.6@us.af.mil', 'phone': '3349537835', 'title': None, 'fullName': 'Brandon Osborne'}, {'fax': '3349532198', 'type': 'secondary', 'email': 'jeremy.kersey.1@us.af.mil', 'phone': '3349533526', 'title': None, 'fullName': 'Jeremy B. Kersey'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ff5e29951ed14b0386ea3291fa6c573f,OFFICE,"{'zipcode': '36112-6334', 'city': 'MAXWELL AFB', 'countryCode': 'USA', 'state': 'AL'}","{'city': {'code': '51000', 'name': 'Montgomery'}, 'state': {'code': 'AL', 'name': 'Alabama'}, 'zip': '36112', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",NaN,https://beta.sam.gov/opp/ff5e29951ed14b0386ea3291fa6c573f/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=ff5e29951ed14b0386ea3291fa6c573f&limit=1', 'hreflang': None, 'media': None, 'title': None, 'type': None, 'deprecation': None}]",['https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/c2a6338838ed4ebfa7d70e91221ffc21/download?api_key=null&token=']
2,ff2a16aad70246d1ba10e639c11456d4,Metrology Equipment Move,SP4703-20-Q-0045,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DCSO RICHMOND,2020-04-27,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2020-05-19,Total Small Business Set-Aside (FAR 19.5),SBA,2020-05-04T15:00:00-04:00,238990.0,Z1PZ,Yes,NaN,"[{'fax': '', 'type': 'primary', 'email': 'Kiara.leslie@dla.mil', 'phone': '8042796566', 'title': None, 'fullName': 'Kiara Leslie'}, {'fax': '', 'type': 'secondary', 'email': 'mary.washington@dla.mil', 'phone': '8042795628', 'title': None, 'fullName': 'Mary A. Washington, Contracting Officer'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ff2a16aad70246d1ba10e639c11456d4,OFFICE,"

In [4]:
df.sort_values(by='postedDate', ascending=False, ignore_index=True, inplace=True)

#### Content Based Recommender System

**using Tf-Idf Vectorizer and cosine similarity**

[source](https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831)

In [5]:
# define function to tokenize a column

def tokenizer_function(column):
    """
    Takes in a text column
        tokenizes the text in each row
        using pattern [[a-zA-Z]\w+]
        which matches every lowercase and upperase character between a-z that are word characters
    Returns list of strings
    """
    
    # instantiate empty list of tokenized text
    texts = []
    
    # instantiate tokenizer
    tokenizer = RegexpTokenizer('[a-zA-Z]\w+')
    
    # create for loop to tokenize each row and add the list of tokens to texts
    for text in column:
        # replace any letter-digit combination with empty string
        text = re.sub(pattern='\w*[0-9]\w*',
                      repl='',
                      string=text)
        # tokenize text
        tokens = tokenizer.tokenize(text)
        
        # transform tokens into lower case strings
        tokens = [token.lower() for token in tokens]
        texts.append(' '.join(tokens))
    return pd.Series(texts)

In [18]:
t = re.sub(pattern='\w*[0-9]\w*',
                      repl='',
                      string='6515--766_MedSurg_VA CMOP Charleston_36C77020Q')

In [15]:
tokenizer_trial = RegexpTokenizer('[a-zA-Z]\w+')

In [21]:
df['title'].head(20)

0                                                                                            ORPI REPLACE CG SEWER LAGOON LINERS
1                                                                              J--DOOR,ACCESS,AIRCRAF, IN REPAIR/MODIFICATION OF
2                                                                              47--MANIFOLD,HYDRAULIC, IN REPAIR/MODIFICATION OF
3                                                                                61--BATTERY ASSEMBLY, IN REPAIR/MODIFICATION OF
4                                                                                                 Spare Parts for MH-65 Aircraft
5                                                                                                        53--MOUNT,RESILIENT,WEA
6     Compact Laser Weapons System (CLaWS) Sustainment Effort in support of Program Manager (PM) Ground Based Air Defense (GBAD)
7                                                                                                

In [19]:
tokenizer_trial.tokenize(t)

['CMOP']

In [6]:
# tokenize title column

tokenized_corpus = tokenizer_function(df['title'])

In [7]:
tokenized_corpus.head()

0                   facility sanitation hvac cleansing
1                                               spe4a6
2    v112 phoenix va carl hayden medical center shu...
3                dairy solicitation new england region
4                       devil elbow dry storage garage
dtype: object

In [8]:
# instantiate Tf-Idf Vectorizer
tvec = TfidfVectorizer(
#     strip_accents='ascii',
    ngram_range=(1, 3), 
    stop_words='english'
#     max_features=500
)

# fit_transform title column
vectorized_matrix = tvec.fit_transform(tokenized_corpus)

# source: https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

In [9]:
tvec.get_feature_names()

['a0',
 'a0 renovate',
 'a0 renovate cwt',
 'a00',
 'a00001',
 'a00001 micro',
 'a00001 micro motion',
 'a00002',
 'a00002 micro',
 'a00002 micro motion',
 'a00009',
 'a00018',
 'a00026',
 'a0003',
 'a00036',
 'a00036 cs',
 'a00036 cs alex',
 'a00037',
 'a00037 veterans',
 'a00037 veterans healthcare',
 'a00039',
 'a00068',
 'a00070',
 'a0008147',
 'a0008147 used',
 'a0008147 used mix',
 'a0008153',
 'a0008153 demil',
 'a0008153 demil mut',
 'a0008160',
 'a0008160 scrap',
 'a0008160 scrap residue',
 'a00087',
 'a00089',
 'a00099',
 'a003',
 'a013',
 'a013 powder',
 'a013 powder road',
 'a01397',
 'a03027',
 'a03028',
 'a04',
 'a04 felix',
 'a04 felix st',
 'a1',
 'a1 leased',
 'a1 leased tail',
 'a1 rm',
 'a1 rm utility',
 'a100',
 'a100 rm',
 'a100 rm n34',
 'a100mb',
 'a100mb ethernet',
 'a100mb ethernet service',
 'a120',
 'a120 digit',
 'a120 digit reference',
 'a135',
 'a135 sports',
 'a135 sports arena',
 'a13627',
 'a13627 l3',
 'a13627 l3 nsn',
 'a141',
 'a141 fl',
 'a141 fl in

In [10]:
# get the vocabulary with their tf-idf indeces
vocab_dict = tvec.vocabulary_

In [11]:
# reverse the vocab_dict - to have indeces as keys, and words as values
inverse_vocab_dict = {value: key for (key, value) in vocab_dict.items()}

In [12]:
# define function that gives back sorted dictionary of indeces and vocabulary 
def dictionary_sort(dictionary):
    """ takes dictionary with indeces (integers or floats) as keys
        returns sorted dictionary
        descending order """
    # isolate the keys of the dictionary
    # sort them in descending order
    sorted_key_list = sorted(dictionary.keys(), reverse=True)
    # instantiate empty dictionary to store key: value pairs
    sorted_dictionary = {}
    # iterate through sorted list of keys
    # add key: value pair to sorted dictionary
    for number in sorted_key_list:
        sorted_dictionary[number] = dictionary[number]
    return sorted_dictionary

In [13]:
# dictionary of title vocabulary
# ordered by their importance relative to the corpus of titles
dictionary_sort(inverse_vocab_dict)

{168980: 'zzyzx head mojave',
 168979: 'zzyzx head',
 168978: 'zzyzx',
 168977: 'zx tm wind',
 168976: 'zx tm',
 168975: 'zx',
 168974: 'zurich instruments equal',
 168973: 'zurich instruments',
 168972: 'zurich',
 168971: 'zuni wastewater treatment',
 168970: 'zuni wastewater',
 168969: 'zuni',
 168968: 'zumwalt ddg lltm',
 168967: 'zumwalt ddg',
 168966: 'zumwalt',
 168965: 'zukeran elementary school',
 168964: 'zukeran elementary',
 168963: 'zukeran',
 168962: 'zshield face cloth',
 168961: 'zshield face',
 168960: 'zshield',
 168959: 'zoom webcast training',
 168958: 'zoom webcast',
 168957: 'zoom platinum level',
 168956: 'zoom platinum',
 168955: 'zoom',
 168954: 'zones minneapolis vamc',
 168953: 'zones minneapolis',
 168952: 'zones matoc',
 168951: 'zones hubzone small',
 168950: 'zones hubzone',
 168949: 'zones',
 168948: 'zone weg subscale',
 168947: 'zone weg hq',
 168946: 'zone weg',
 168945: 'zone vessel traffic',
 168944: 'zone vessel',
 168943: 'zone traffic calming',
 1

In [14]:
# check object type of vectorized matrix
type(vectorized_matrix)

scipy.sparse.csr.csr_matrix

In [17]:
# calculate matrix of cosine similarities
recommender = pairwise_distances(vectorized_matrix, metric='cosine')

In [18]:
recommender.shape

(43454, 43454)

In [19]:
df.head(2)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,...,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
0,00d3d2a3ab47486894a35ba3302e8c8c,Facility Sanitation HVAC Cleansing,NMAN7100-20-00736,"COMMERCE, DEPARTMENT OF",NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION,DEPT OF COMMERCE NOAA,2020-05-29,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,...,NaN,"[{'fax': '', 'type': 'primary', 'email': 'Chri...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '64106', 'city': 'KANSAS CITY', 'c...","{'city': {'code': '57000', 'name': 'Norfolk'},...",NaN,https://beta.sam.gov/opp/00d3d2a3ab47486894a35...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",['https://beta.sam.gov/api/prod/opps/v3/opport...
1,61dbf3bcbe114b038365e449aaf0ae80,SPE4A6-20-R-0090,SPE4A6-20-R-0090,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA AVIATION,2020-05-29,Justification,Justification,auto30,...,"{'date': '2020-05-29', 'number': 'SPE4A6-20-D-...","[{'fax': '', 'type': 'primary', 'email': 'KYON...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '23297', 'city': 'RICHMOND', 'coun...",NaN,NaN,https://beta.sam.gov/opp/61dbf3bcbe114b038365e...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",['https://beta.sam.gov/api/prod/opps/v3/opport...


In [20]:
# create recommender dataframe
recommender_df = pd.DataFrame(recommender,
                              index=df['title'],
                              columns=df['title'])

In [22]:
recommender_df.head()

title,Facility Sanitation HVAC Cleansing,SPE4A6-20-R-0090,V112--Phoenix VA Carl T. Hayden Medical Center Shuttle Bus Services,Dairy Solicitation - New England Region,Y--DEVIL'S ELBOW DRY STORAGE GARAGE,"6515-- Stock-Tables,Mattresses,Beds,etc.","Gulf Intracoastal Waterway, Texas, Channel to Victoria Middle and Lower Reach in Calhoun and Victoria Counties, Texas, Maintenance Pipeline Dredging",Remanufacture/Upgrade/Recore Heat Exchangers,"42--GOGGLES,INDUSTRIAL",NOVASTAR Source Sought - NASIC S&TI Services,...,Weapons and Systems Integration Support Services,"AWARD A 1GB ETHERNET CONNECTION (RATE LIMIT TO 500MB) FROM 1 LINSLEY DR, PLAINVILLE, CT 06062 TO 77 GRAYLING AVENUE, NEW LONDON NAVAL SUBMARINE BASE, GROTON, CT 06349.","Award of a 1 GB ETHERNET FROM (BLDG) 261; (RM) 211; (FL) 2ND; 2308 FLORIDA KEYS AVE, TAMPA, FL, 33621, US TO (BLDG) 500; (RM) 703; (FL) 7; 500 EAST ZACK ST; TAMPA, FL, 33602.",Dry Goods/Meats 2nd Qtr FY20,"SUBMIT A QUOTE TO PROVIDE, INSTALL, AND MAINTAIN A 1GB ETHERNET SERVICE AT (BLDG) 1415; (RM) BCO/TELCO; 7177 LANGLEY STREET, MILTON, FL 32570 AND BLDG 90215; DCO ROOM; 306 CODY AVENUE, HURLBURT FIELD, FL 32544.",Purchase of Gearbox Assembly,COHO,J--REPLACE VICTORIAN PARK LIGHT FIXTURES AT: SF Marit,"PROVIDE, INSTALL, AND MAINTAIN A 2.488GB (OC48C) BETWEEN BLDG 3782, RM174, FL: 1, 1000 CHIEF’S WAY, CORRY STATION, PENSACOLA, FL 32511, AND BLDG 9357, RM 116, FL: 1, 8357 CYPRESS LOOP RD., STENNIS SPACE CENTER, MS 39529","SUBMIT A QUOTE TO PROVIDE, INSTALL, AND MAINTAIN A 10GBE (GIGABIT ETHERNET) OVER OTN (OTU-2) COMMERCIAL LEASE BETWEEN NAS JOINT RESERVE BASE NEW ORLEANS 400 RUSSELL AVENUE, NEW ORLEANS, LA, AND 3326 GENERAL HUDNELL DRIVE, SAN ANTONIO, TX."
title,,,,,,,,,,,,,,,,,,,,,
Facility Sanitation HVAC Cleansing,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000
SPE4A6-20-R-0090,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000
V112--Phoenix VA Carl T. Hayden Medical Center Shuttle Bus Services,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.992358,...,0.990194,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,0.992956,1.000000
Dairy Solicitation - New England Region,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.000000,0.987203,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,0.976354
Y--DEVIL'S ELBOW DRY STORAGE GARAGE,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.000000,1.000000,1.0,0.958484,1.0,1.0,1.0,1.0,1.000000,1.000000


In [64]:
# recommender function
def search(search_term):
    solicitations = df[df['title'].str.contains(search_term)]['title']
    for solicitation in solicitations:
        print('Search Term:', search_term)
        print('')
        print(recommender_df[solicitation].sort_values(ascending=False))
        
        return df[df['title'] == solicitation]
        
# source: Riley Dallas - lesson 8.06

In [65]:
search('Garage')

Search Term: Garage

title
SUBMIT A QUOTE TO PROVIDE, INSTALL, AND MAINTAIN A 10GBE (GIGABIT ETHERNET) OVER OTN (OTU-2) COMMERCIAL LEASE BETWEEN NAS JOINT RESERVE BASE NEW ORLEANS 400 RUSSELL AVENUE, NEW ORLEANS, LA, AND 3326 GENERAL HUDNELL DRIVE, SAN ANTONIO, TX.    1.000000
VALVE/ACTUATOR ASSY                                                                                                                                                                                                                               1.000000
25--WHEEL,PNEUMATIC TIR                                                                                                                                                                                                                           1.000000
15--FITTING,STRUCTURAL                                                                                                                                                                                                      

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,...,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
413,de2801a31589455cb35cc1d938eec09a,C1LZ--Parking Garage Design,36E77620R0028,"VETERANS AFFAIRS, DEPARTMENT OF","VETERANS AFFAIRS, DEPARTMENT OF",PCAC NATIONAL ENERGY BUSINESS CENTER (36E776),2020-05-29,Presolicitation,Presolicitation,autocustom,...,{'awardee': {}},"[{'fax': '', 'type': 'primary', 'email': 'doug...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '44131', 'city': 'INDEPENDENCE', '...",{'streetAddress': ' VA Connecticut Healthcare ...,NaN,https://beta.sam.gov/opp/de2801a31589455cb35cc...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",['https://beta.sam.gov/api/prod/opps/v3/opport...


In [53]:
search_term = 'dairy'
titles = df['title'].str.lower()
titles[1].contains(search_term)
# for title in titles:
#     df.loc[title.contains(search_term)]
# # df.loc[title.contains(search_term) for title in titles]['title']
# solicitations
# for solicitation in solicitations:
#     print(recommender_df[solicitation].sort_values(by=solicitation))

AttributeError: 'str' object has no attribute 'contains'

In [32]:
# cosine similarity matrix
# add some other columns into a bag of words
# further clean the titles

# figure out how to call up titles that were similar to a search term
# try defining max_df instead of removing english stop words
# min_df as well - if something occures just once 
# maybe I need 2-grams to 4-grams (2, 4)

[168980,
 168979,
 168978,
 168977,
 168976,
 168975,
 168974,
 168973,
 168972,
 168971,
 168970,
 168969,
 168968,
 168967,
 168966,
 168965,
 168964,
 168963,
 168962,
 168961,
 168960,
 168959,
 168958,
 168957,
 168956,
 168955,
 168954,
 168953,
 168952,
 168951,
 168950,
 168949,
 168948,
 168947,
 168946,
 168945,
 168944,
 168943,
 168942,
 168941,
 168940,
 168939,
 168938,
 168937,
 168936,
 168935,
 168934,
 168933,
 168932,
 168931,
 168930,
 168929,
 168928,
 168927,
 168926,
 168925,
 168924,
 168923,
 168922,
 168921,
 168920,
 168919,
 168918,
 168917,
 168916,
 168915,
 168914,
 168913,
 168912,
 168911,
 168910,
 168909,
 168908,
 168907,
 168906,
 168905,
 168904,
 168903,
 168902,
 168901,
 168900,
 168899,
 168898,
 168897,
 168896,
 168895,
 168894,
 168893,
 168892,
 168891,
 168890,
 168889,
 168888,
 168887,
 168886,
 168885,
 168884,
 168883,
 168882,
 168881,
 168880,
 168879,
 168878,
 168877,
 168876,
 168875,
 168874,
 168873,
 168872,
 168871,
 168870,
 

To do list:

1. further clean the texts
2. build the cosin similarities matrix
3. include other columns into the cosin similarities matrix
4. right the function that returns the similar titles